In [1]:
import pandas as pd
import sqlite3
from recipe_query import query_recipes
from dash import Dash, html, dcc, ctx
from dash.dependencies import Input, Output, State

## set up `ingredients` table

In [2]:
# Global database connection variable
recipes_db = None

def get_recipes_db():
    """Ensures a global database connection is available and returns it, setting it up if not already configured."""
    global recipes_db

    if recipes_db is not None: # if database is not empty
        return recipes_db
    else:
        # Connect to the database recipes_db.sqlite
        recipes_db = sqlite3.connect("recipes_db.sqlite", check_same_thread=False)

        # SQL command to create an `ingredients` table in the database if it does not exist
        cmd = '''
        CREATE TABLE IF NOT EXISTS ingredients (
            ingredient TEXT NOT NULL UNIQUE
        );
        '''
        cursor = recipes_db.cursor()
        cursor.execute(cmd)
        recipes_db.commit()  # saves changes
        cursor.close()  # closes cursor

    return recipes_db

## Add `recipes` table to database

In [3]:
def setup_database():
    """ Imports data from CSV to SQL table using the global connection. """
    db = get_recipes_db()

    # Load data from a CSV file into a DataFrame
    recipes = pd.read_csv("../datasets/cleaned_recipes.csv")
    # Insert data from DataFrame to the SQL table, replacing if exists
    recipes.to_sql("recipes", db, if_exists="replace", index=False)

    # Verification query to check the tables in the database
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    cursor.close()

    return tables

# Call setup_database function and print the tables to verify
tables = setup_database()
print(tables)

[('ingredients',), ('recipes',)]


In [4]:
def insert_ingredient(ingredient):
    """
    Inserts a new ingredient into the database.
    Args:
        ingredient (str): The name of an ingredient.
    """

    # creating a cursor to our database

    db = get_recipes_db()
    cursor = db.cursor()
    try:
        cursor.execute("INSERT INTO ingredients (ingredient) VALUES (?)", (ingredient,))
        db.commit()
        return True  # Insertion was successful
    except sqlite3.IntegrityError:
        return False  # Insertion failed due to duplicate ingredient
    finally:
        cursor.close()

def fetch_ingredients():
    """ Fetches all ingredients from the database to display them """
    db = get_recipes_db()
    cursor = db.cursor()
    cursor.execute("SELECT ingredient FROM ingredients")
    ingredients = cursor.fetchall()
    cursor.close()
    return ingredients

In [5]:
# creating dash app
app = Dash(__name__)

#defining the layout
app.layout = html.Div([
    # Title
    html.H1("No-Plan Pantry", style={'textAlign': 'center', 'color': 'pink'}),

    # Description
    html.P(
        "Discover a new way to minimize food waste and unlock your inner chef. This application is designed to help you create delicious, easy-to-make recipes using leftover ingredients from your fridge. Simply input the items you have on hand, and let our app suggest creative meal ideas that are both tasty and resourceful. Whether you're looking to save time, reduce waste, or explore new recipes, this tool is here to inspire your cooking adventures. Give it a try and turn your leftovers into something delicious!", 
        style={'textAlign': 'center', 'fontSize': '18px'}
    ),

    # Ingredients Section
    html.Div([
        html.H2("Ingredients:", style={'textAlign': 'left', 'marginTop': '30px'}),

        # First row: Vegetables (left), Proteins (center), Fruits (right)
        html.Div([
            # Vegetables
            html.Div([
                html.H3("Vegetables"),
                html.Div([
                    html.Div([
                        html.Span("Lettuce", style={'marginRight': '10px'}),
                        html.Button("Add", id="lettuce-btn")
                    ]),
                    html.Div([
                        html.Span("Cabbage", style={'marginRight': '10px'}),
                        html.Button("Add", id="cabbage-btn")
                    ]),
                    html.Div([
                        html.Span("Cucumbers", style={'marginRight': '10px'}),
                        html.Button("Add", id="cucumbers-btn")
                    ]),
                ], style={'marginLeft': '20px'}),
            ], style={'flex': '1', 'textAlign': 'left'}),

            # Proteins
            html.Div([
                html.H3("Proteins"),
                html.Div([
                    html.Div([
                        html.Span("Chicken", style={'marginRight': '10px'}),
                        html.Button("Add", id="chicken-btn")
                    ]),
                    html.Div([
                        html.Span("Beef", style={'marginRight': '10px'}),
                        html.Button("Add", id="beef-btn")
                    ]),
                    html.Div([
                        html.Span("Tofu", style={'marginRight': '10px'}),
                        html.Button("Add", id="tofu-btn")
                    ]),
                ], style={'marginLeft': '20px'}),
            ], style={'flex': '1', 'textAlign': 'center'}),

            # Fruits
            html.Div([
                html.H3("Fruits"),
                html.Div([
                    html.Div([
                        html.Span("Apple", style={'marginRight': '10px'}),
                        html.Button("Add", id="apple-btn")
                    ]),
                    html.Div([
                        html.Span("Banana", style={'marginRight': '10px'}),
                        html.Button("Add", id="banana-btn")
                    ]),
                    html.Div([
                        html.Span("Grapes", style={'marginRight': '10px'}),
                        html.Button("Add", id="grape-btn")
                    ]),
                ], style={'marginLeft': '20px'}),
            ], style={'flex': '1', 'textAlign': 'right'}),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '30px'}),

        # Second row: Grains (left), Dairy (center), Condiments (right)
        html.Div([
            # Grains
            html.Div([
                html.H3("Grains"),
                html.Div([
                    html.Div([
                        html.Span("White Rice", style={'marginRight': '10px'}),
                        html.Button("Add", id="white_rice-btn")
                    ]),
                    html.Div([
                        html.Span("Oats", style={'marginRight': '10px'}),
                        html.Button("Add", id="oats-btn")
                    ]),
                    html.Div([
                        html.Span("Barley", style={'marginRight': '10px'}),
                        html.Button("Add", id="barley-btn")
                    ]),
                ], style={'marginLeft': '20px'}),
            ], style={'flex': '1', 'textAlign': 'left'}),

            # Dairy
            html.Div([
                html.H3("Dairy"),
                html.Div([
                    html.Div([
                        html.Span("Milk", style={'marginRight': '10px'}),
                        html.Button("Add", id="milk-btn")
                    ]),
                    html.Div([
                        html.Span("Cheese", style={'marginRight': '10px'}),
                        html.Button("Add", id="cheese-btn")
                    ]),
                    html.Div([
                        html.Span("Yogurt", style={'marginRight': '10px'}),
                        html.Button("Add", id="yogurt-btn")
                    ]),
                ], style={'marginLeft': '20px'}),
            ], style={'flex': '1', 'textAlign': 'center'}),

            # Condiments
            html.Div([
                html.H3("Condiments"),
                html.Div([
                    html.Div([
                        html.Span("Ketchup", style={'marginRight': '10px'}),
                        html.Button("Add", id="ketchup-btn")
                    ]),
                    html.Div([
                        html.Span("Mustard", style={'marginRight': '10px'}),
                        html.Button("Add", id="mustard-btn")
                    ]),
                    html.Div([
                        html.Span("Mayo", style={'marginRight': '10px'}),
                        html.Button("Add", id="mayo-btn")
                    ]),
                ], style={'marginLeft': '20px'}),
            ], style={'flex': '1', 'textAlign': 'right'}),
        ], style={'display': 'flex', 'justifyContent': 'space-between'})
    ]),

    # Additional Ingredients
    html.Div([
        html.H2("Additional Ingredients:", style={'marginTop': '40px'}),
        html.Div([
            dcc.Input(
                id='new-ingredient-input', 
                type='text', 
                placeholder='Enter ingredient name', 
                style={'width': '300px', 'marginRight': '10px'}
            ),
            html.Button("Add Ingredient", id="add-ingredient-btn", style={
                'backgroundColor': 'pink', 
                'border': 'none', 
                'color': 'white',
                'padding': '5px 10px',
                'cursor': 'pointer'
            })
        ], style={'display': 'flex', 'alignItems': 'center', 'marginTop': '10px'})
    ]),

    # Display Added Ingredients Section
    html.Div(id='ingredients-list', style={'marginTop': '20px', 'textAlign': 'center'}),  # Ensure this DIV is part of the layout

    # Time Limit Section
    html.Div([
        html.H2("Time Limit (Cook & Prep Time):", style={'marginTop': '40px'}),
        dcc.Slider(
            id='time-limit-slider',
            min=0,
            max=60,
            step=1,
            marks={i: f"{i} min" for i in range(0, 61, 10)},  # Add marks at 10-minute intervals
            value=30,  # Default value
            tooltip={"placement": "bottom", "always_visible": True}
        ),
        html.Button("Submit", id="submit-btn", style={
            'marginTop': '20px',
            'backgroundColor': 'pink', 
            'color': 'white',
            'border': 'none',
            'padding': '10px 20px',
            'fontSize': '16px',
            'cursor': 'pointer',
            'display': 'block',
            'marginLeft': 'auto',
            'marginRight': 'auto'
        }),
    ], style={'marginTop': '20px', 'marginBottom': '40px'}),

    # Results Section
    html.Div([
        html.H2("Results:", style={'marginTop': '40px', 'textAlign': 'center'}),
        html.Div([
            # Matched Recipes
            html.Div([
                html.H3("Matched Recipes"),
                html.Ul([
                    html.Li("Chicken Nuggets"),
                    html.Li("Chicken Salad"),
                    html.Li("Chicken Noodle Soup"),
                ]),
            ], style={'flex': '1', 'textAlign': 'left', 'marginLeft': '20px'}),

            # AI-Generated Recipes
            html.Div([
                html.H3("AI-Generated Recipes"),
                html.Ul([
                    html.Li("Chicken Pho"),
                    html.Li("Fried Chicken"),
                    html.Li("Chicken Fried Rice"),
                ], style={'marginLeft': '0'})  # Ensure bullet points align with the header
            ]),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '30px'})
    ]),

    # Section to display all submitted ingredients
    html.H3("Submitted Ingredients:"),
    html.Ul(id="display-ingredients", style={'marginTop': '10px', 'textAlign': 'left'}),
])


@app.callback(
    [Output('ingredients-list', 'children'),
     Output('display-ingredients', 'children')],
    [Input(f'{item}-btn', 'n_clicks') for item in [
        'lettuce', 'cabbage', 'cucumbers', 'chicken', 'beef', 'tofu',
        'apple', 'banana', 'grape', 'white_rice', 'oats', 'barley',
        'milk', 'cheese', 'yogurt', 'ketchup', 'mustard', 'mayo'
    ]] + [Input('add-ingredient-btn', 'n_clicks')],
    [State('new-ingredient-input', 'value')],
    prevent_initial_call=True
)
def update_ingredients_list(*args):
    """ Updates ingredients table based on user input"""
    triggered_id = ctx.triggered_id
    ingredient = triggered_id.split('-')[0] if triggered_id != 'add-ingredient-btn' else ctx.states['new-ingredient-input.value']
    
    # Attempt to insert the ingredient
    success = insert_ingredient(ingredient)
    
    # Fetch the updated list of ingredients to display
    ingredients = fetch_ingredients()
    list_items = [html.Li(ingredient[0]) for ingredient in ingredients]
    
    # Create feedback message
    feedback_message = f"Added: {ingredient}" if success else f"Duplicate not added: {ingredient}"
    return feedback_message, list_items

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

## Getting Recommended Recipes Based on User Input

In [6]:
query = "SELECT * FROM ingredients"

# Read the query results into a DataFrame
df = pd.read_sql_query(query, recipes_db)
ings = [df["ingredient"][i] for i in range(len(df))]
ings

['milk', 'cheese', 'chicken', 'beef', 'tofu', 'cabbage']

In [17]:
df = query_recipes("recipes_db.sqlite", 
                   ingredients=ings
                   )
df.head(3)

,recipe_name,category_name,rating,prep_time,cook_time,additional_time,total_time,num_servings_per_recipe,ingredients_list,direction_list,...,fiber (g),sugar (g),cholesterol (mg),vitamin_c (mg),calcium (mg),iron (mg),potassium (mg),recipe_link,main_ingredients,main_ing_len
10110,Chicken Alfredo with Fettuccini Noodles,Fettuccini,3.8,20.0,15.0,0.0,35.0,8,"fettuccini pasta,butter, divided,skinless, bon...",Bring a large pot of lightly salted water to ...,...,2.0,2.0,272.0,1.0,410.0,3.0,436.0,https://www.allrecipes.com/recipe/26258/chicke...,"[cheese, chicken, milk]",3
6023,Polenta Fries,Polenta,4.1,20.0,20.0,120.0,160.0,8,"whole milk,chicken stock,butter,dry polenta,Pa...","In a large saucepan, combine milk, chicken st...",...,4.0,6.0,29.0,22.0,302.0,6.0,116.0,https://www.allrecipes.com/recipe/64240/polent...,"[cheese, chicken, milk]",3
3043,Sloppy Joes I,Sloppy Joes,4.5,10.0,60.0,0.0,70.0,6,"lean ground beef,condensed chicken gumbo soup,...","In large skillet over medium heat, brown meat...",...,1.0,2.0,58.0,3.0,18.0,2.0,251.0,https://www.allrecipes.com/recipe/24262/sloppy...,"[beef, chicken]",2
